# Introduction

This is an exploration of autoencoders for representing gene expression data. The main question here is to to explore whether beta autoencoders better capture the variance in the data than vanilla autoencoders. This analysis will test the findings of Chow et al. on a different dataset.  

## Setting up the environment
* Specifically, I am setting up the analysis to be run in a GPU environment, if a GPU is present. 


In [116]:
import sys
import pathlib
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import matplotlib.pyplot as plt

import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


output_dir = pathlib.Path("data")
output_dir.mkdir(exist_ok=True)


cuda:0


## Loading in the data
Beginning with the data from GSEA-InContext


In [37]:
# load the data
# GSEA-InContext Data
full_data = pd.read_csv("data/all_expression_data_geneLevel.csv",  index_col = 0)
sample_annotations = pd.read_csv("data/all_experiments_sample_annotations.csv")

In [68]:
print("sample shape", sample_annotations.shape)
print("full data shape", full_data.shape)
# columns are samples, rows are genes



sample shape (104851, 20)
full data shape (7381, 2410)
['keep' 'drug' 'tissue_type' 'cell_type' 'cell_line' 'vehicle' 'notes'
 'gse' 'gse_title' 'gse_summary' 'gsm' 'comparison' 'sample_group'
 'gsm_title' 'gsm_description' 'query_vehicle' 'query_drug' 'query_treat'
 'query_sirna' 'query_antibody']


In [ ]:
# first column is probe, need to drop
full_data = full_data.drop("Probe", 1)

In [60]:
# Transpose the data so that the rows are subjects
full_data_t = full_data.T

In [80]:
# Filter metadata by only kept features
sample_annotations = sample_annotations[sample_annotations.gsm.isin(full_data_t.index.values)]
sample_annotations.shape



(2410, 20)

## Splitting the data
Splitting the data into train, test, and validation datasets (70%,15%,15%). 

Do I need to consider the experiment assignments in splitting the data?
I think so, currently stratifying by sample_group variable. This should ensure that the train and test groups have similar amounts of experiment and control data. 

In [87]:
test_split = .3
seed = 42

# Split data
train_df, test_df = train_test_split(
    full_data_t,
    test_size=test_split,
    random_state=seed,
    stratify = sample_annotations.sample_group,
)

test_df, valid_df = train_test_split(
    test_df,
    test_size=0.5,
    random_state=seed
)


In [88]:
print(train_df.shape)
print(test_df.shape)
print(valid_df.shape)

(1687, 7381)
(361, 7381)
(362, 7381)


## Normalization 
* Normalizing each dataset separately to avoid data leakage from one dataset to another.
* Using Minmax transformatin[0,1]

In [138]:
min_max_scaler = preprocessing.MinMaxScaler()
minmax_train = pd.DataFrame(min_max_scaler.fit_transform(train_df), 
                            columns = train_df.columns.values,
                            index = train_df.index.values)
minmax_test = pd.DataFrame(min_max_scaler.fit_transform(test_df), 
                            columns = test_df.columns.values,
                            index = test_df.index.values)

minmax_valid = pd.DataFrame(min_max_scaler.fit_transform(valid_df), 
                            columns = valid_df.columns.values,
                            index = valid_df.index.values)




In [140]:
# Output data splits
train_file = pathlib.Path(output_dir, "GSEA_InContext_train.tsv.gz")
test_file = pathlib.Path(output_dir, "GSEA_InContext_test.tsv.gz")
valid_file = pathlib.Path(output_dir, "GSEA_InContext_valid.tsv.gz")
complete_file = pathlib.Path(output_dir, "GSEA_InContext_complete.tsv.gz")

minmax_train.to_csv(train_file, sep="\t", index=False, float_format="%.5g")
minmax_test.to_csv(test_file, sep="\t", index=False, float_format="%.5g")
minmax_valid.to_csv(valid_file, sep="\t", index=False, float_format="%.5g")
#full_data_t.to_csv(complete_file, sep="\t", index=False, float_format="%.5g")

Code below is repeating a tutorial to learn how the autoencoder models are trained. 

In [4]:
# Import libraries
import torchvision                             # contains image datasets and many functions to manipulate images
import torchvision.transforms as transforms    # to normalize, scale etc the dataset
from torch.utils.data import DataLoader        # to load data into batches (for SGD)
from torchvision.utils import make_grid        # Plotting. Makes a grid of tensors
from torchvision.datasets import MNIST         # the classic handwritten digits dataset
import matplotlib.pyplot as plt                # to plot our images
import numpy as np

# Create Dataset object.s Notice that ToTensor() transforms images to pytorch
# tensors AND scales the pixel values to be within [0, 1]. Also, we have separate Dataset
# objects for training and test sets. Data will be downloaded to a folder called 'data'.
trainset = MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
testset  = MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

# Create DataLoader objects. These will give us our batches of training and testing data.
batch_size = 100
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
testloader  = DataLoader(testset, batch_size=batch_size, shuffle=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

9920512it [00:00, 10225594.92it/s]                            


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


32768it [00:00, 523252.05it/s]
0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


1654784it [00:00, 3316268.05it/s]                             
0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


8192it [00:00, 48368.86it/s]            

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


In [5]:
import torch.nn as nn                          # Class that implements a model (such as a Neural Network)
import torch.nn.functional as F                # contains activation functions, sampling layers etc
import torch.optim as optim   

In [6]:
e_hidden = 500        # Number of hidden units in the encoder. See AEVB paper page 7, section "Marginal Likelihood"
d_hidden = 500        # Number of hidden units in the decoder. See AEVB paper page 7, section "Marginal Likelihood"
latent_dim = 2        # Dimension of latent space. See AEVB paper, page 7, section "Marginal Likelihood"
learning_rate = 0.001 # For optimizer (SGD or Adam)
weight_decay = 1e-5   # For optimizer (SGD or Adam)
epochs = 50           # Number of sweeps through the whole dataset

In [7]:
class VAE(nn.Module):
    def __init__(self):
        """Variational Auto-Encoder Class"""
        super(VAE, self).__init__()
        # Encoding Layers
        self.e_input2hidden = nn.Linear(in_features=784, out_features=e_hidden)
        self.e_hidden2mean = nn.Linear(in_features=e_hidden, out_features=latent_dim)
        self.e_hidden2logvar = nn.Linear(in_features=e_hidden, out_features=latent_dim)
        
        # Decoding Layers
        self.d_latent2hidden = nn.Linear(in_features=latent_dim, out_features=d_hidden)
        self.d_hidden2image = nn.Linear(in_features=d_hidden, out_features=784)
        
    def forward(self, x):
        # Shape Flatten image to [batch_size, input_features]
        x = x.view(-1, 784)
        
        # Feed x into Encoder to obtain mean and logvar
        x = F.relu(self.e_input2hidden(x))
        mu, logvar = self.e_hidden2mean(x), self.e_hidden2logvar(x)
        
        # Sample z from latent space using mu and logvar
        if self.training:
            z = torch.randn_like(mu).mul(torch.exp(0.5*logvar)).add_(mu)
        else:
            z = mu
        
        # Feed z into Decoder to obtain reconstructed image. Use Sigmoid as output activation (=probabilities)
        x_recon = torch.sigmoid(self.d_hidden2image(torch.relu(self.d_latent2hidden(z))))
        
        return x_recon, mu, logvar

In [10]:
# Loss
def vae_loss(image, reconstruction, mu, logvar):
  """Loss for the Variational AutoEncoder."""
  # Binary Cross Entropy for batch
  #BCE = F.binary_cross_entropy(input=reconstruction.view(-1, 28*28), target=image.view(-1, 28*28), reduction='sum')
  # Mean Square Error for batch
  MSE = F.mse_loss(input=reconstruction.view(-1, 28*28), target=image.view(-1, 28*28), reduction='mean')
  # Closed-form KL Divergence
  KLD = 0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
  #return BCE - KLD
  return MSE - KLD

# Instantiate VAE with Adam optimizer
vae = VAE()
vae = vae.to(device)    # send weights to GPU. Do this BEFORE defining Optimizer
optimizer = optim.Adam(params=vae.parameters(), lr=learning_rate, weight_decay=weight_decay)
vae.train()            # tell the network to be in training mode. Useful to activate Dropout layers & other stuff

# Train
losses = []

for epoch in range(epochs):
  # Store training losses & instantiate batch counter
  losses.append(0)
  number_of_batches = 0

  # Grab the batch, we are only interested in images not on their labels
  for images, _ in trainloader:
    # Save batch to GPU, remove existing gradients from previous iterations
    images = images.to(device)
    optimizer.zero_grad()

    # Feed images to VAE. Compute Loss.
    reconstructions, latent_mu, latent_logvar = vae(images)
    loss = vae_loss(images, reconstructions, latent_mu, latent_logvar)

    # Backpropagate the loss & perform optimization step with such gradients
    loss.backward()
    optimizer.step()

    # Add loss to the cumulative sum
    losses[-1] += loss.item()  
    number_of_batches += 1
  
  # Update average loss & Log information
  losses[-1] /= number_of_batches
  print('Epoch [%d / %d] average reconstruction error: %f' % (epoch+1, epochs, losses[-1]))    

Epoch [1 / 50] average reconstruction error: 0.132250
Epoch [2 / 50] average reconstruction error: 0.068348
Epoch [3 / 50] average reconstruction error: 0.067769
Epoch [4 / 50] average reconstruction error: 0.067607
Epoch [5 / 50] average reconstruction error: 0.067559
Epoch [6 / 50] average reconstruction error: 0.067535
Epoch [7 / 50] average reconstruction error: 0.067523
Epoch [8 / 50] average reconstruction error: 0.067513
Epoch [9 / 50] average reconstruction error: 0.067500
Epoch [10 / 50] average reconstruction error: 0.067493


KeyboardInterrupt: 